从原始的json数据生成patient维度数据  

In [1]:
import json
import pandas as pd
import numpy as np
import datetime
import pickle

# data preparation

In [2]:
raw_data = pd.read_json("/Users/weiyuna/Desktop/工作/合作项目/医大一-胃癌/模型/code_202203/data/fuzhu.json")
data = raw_data.copy()
data
data.to_csv("./data/data_fuzhu.csv", index = False)

## preprocessing

In [3]:
data["性别"] = data["人口学_性别"].map(lambda x: 1 if x == "男" else 0)

yes_no_list = [
    'C0*既往化疗',
    '治疗史_既往放疗',
    
    '合并慢性疾病_充血性心力衰竭',
    '合并慢性疾病_心绞痛',
    '合并慢性疾病_心肌梗死',
    '合并慢性疾病_慢阻肺',
    '合并慢性疾病_消化性溃疡',
    '合并慢性疾病_甲状腺疾病',
    '合并慢性疾病_糖尿病',
    '合并慢性疾病_自身免疫性疾病',
    '合并慢性疾病_获得性免疫缺陷综合征',
]
for feature_name in yes_no_list:
    data[feature_name] = data[feature_name].map(lambda x: 1 if x == "是" else 0)

In [4]:
# 合并肿瘤状态
status_feature_list = [
    '肿瘤状态_pM分期',
    '肿瘤状态_pN分期',
    '肿瘤状态_pT分期'
]
for status_feature in status_feature_list:
    data[status_feature] = data[status_feature].map(lambda x: x[:3] if x is not None else x)

In [5]:
# 计算周期年份 便于计算训练集和测试集
data["年份"] = data["C1"].map(lambda x: datetime.datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S").year)

In [6]:
# 检验指标的动态变量处理为上周期检验结果的最低值
lab_feature = [
    '血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血',
    '血常规血生化_生化检查:前白蛋白(PA)-静脉血',
    '血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血',
    '血常规血生化_生化检查:总胆红素(TBIL)-静脉血',
    '血常规血生化_生化检查:白蛋白(ALB)-静脉血',
    '血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血',
    '血常规血生化_生化检查:肌酐(Crea)-静脉血',
    '血常规血生化_血常规:中性粒细胞计数-静脉血',
    '血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血',
    '血常规血生化_血常规:白细胞计数(WBC#)-静脉血',
    '血常规血生化_血常规:血小板计数(PLT#)-静脉血',
    '血常规血生化_血常规:血红蛋白(Hb)-静脉血']
for cycle in range(1,9):
    for feature in lab_feature:
        column_name = "C"+str(cycle)+"*"+feature
        last_column_name = "C"+str(cycle-1)+"*"+feature
        data[column_name] = raw_data[last_column_name].map(lambda x: min(x) if len(x)>0 else np.nan)

In [7]:
# 预防用药处理为上周期用药的布尔值
for cycle in range(1,9):
    column_name = "C"+str(cycle)+"*预防性抗骨髓抑制药物"
    last_column_name = "C"+str(cycle-1)+"*预防性抗骨髓抑制药物"
    if cycle == 1:
        data[column_name] = 0
    else:
        data[column_name] = raw_data[last_column_name].map(lambda x: 0 if len(x) == 0 else 1)

In [8]:
# 肾小球滤过率和ps评分对应上周期的
for cycle in range(1,9):
    column_name = "C"+str(cycle)+"*肾小球滤过率"
    data["C"+str(cycle)+"*肾小球滤过率"] = raw_data["C"+str(cycle-1)+"*肾小球滤过率"]
    data["C"+str(cycle)+"*身体状况评价_PS评分"] = raw_data["C"+str(cycle-1)+"*身体状况评价_PS评分"]

## cycle interval

In [9]:
interval_result = list()
for i in range(len(data)):
    event = data["event"][i]
    cycle_time_list = data["cycle_info"][i]
    interval_list = list()
    if event == 0:
        time_list = cycle_time_list[:9]
        
    else:
        event_cycle = data["首次不良反应#周期"][i]
        event_time = data["首次不良反应#时间"][i]
        time_list = cycle_time_list[:int(event_cycle)]
        time_list.append(event_time)
    
    for i in range(1, len(time_list)):
        j = i-1
        interval = datetime.datetime.strptime(time_list[i], "%Y-%m-%d %H:%M:%S") - datetime.datetime.strptime(time_list[j], "%Y-%m-%d %H:%M:%S")
        interval_list.append(interval.days)
    
    if event == 0 and len(interval_list) < 8:
        interval_list.append(45)
    
    if len(interval_list) < 8:
        interval_list = interval_list + [np.nan for i in range(8 -len(interval_list))]

    interval_result.append(interval_list)

interval_columns = ["C"+str(num)+"*interval" for num in range(1,9)]
t1 = pd.DataFrame(interval_result).apply(pd.Series)
t1.columns = interval_columns
data = data.join(t1)

In [10]:
pd.set_option("max_columns", None)
data

,patient_sn,event,首次不良反应#周期,首次不良反应#时间,人口学_住院号,人口学_出生日期,年龄,人口学_性别,人口学_首诊年龄(岁),体重,身高,ABS,BMI,合并慢性疾病_充血性心力衰竭,合并慢性疾病_心绞痛,合并慢性疾病_心肌梗死,合并慢性疾病_慢阻肺,合并慢性疾病_消化性溃疡,合并慢性疾病_甲状腺疾病,合并慢性疾病_糖尿病,合并慢性疾病_自身免疫性疾病,合并慢性疾病_获得性免疫缺陷综合征,治疗史_既往放疗,肿瘤状态_cM分期,肿瘤状态_cN分期,肿瘤状态_cT分期,肿瘤状态_pM分期,肿瘤状态_pN分期,肿瘤状态_pT分期,肿瘤状态_临床分期,肿瘤状态_病理分期,肿瘤状态_胃部肿瘤位置,cycle_info,cycle_num,C0*既往化疗,C0*既往胃癌手术,C0*肾小球滤过率,C0*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C0*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C0*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C0*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C0*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C0*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C0*血常规血生化_生化检查:肌酐(Crea)-静脉血,C0*血常规血生化_血常规:中性粒细胞计数-静脉血,C0*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C0*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C0*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C0*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C0*身体状况评价_PS评分,C0*预后影响指标,C1,C1*不良反应,C1*不良反应#时间,C1*化疗药物,C1*放射治疗次数,C1*治疗性抗骨髓抑制药物,C1*肾小球滤过率,C1*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C1*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C1*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C1*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C1*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C1*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C1*血常规血生化_生化检查:肌酐(Crea)-静脉血,C1*血常规血生化_血常规:中性粒细胞计数-静脉血,C1*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C1*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C1*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C1*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C1*身体状况评价_PS评分,C1*预防性抗骨髓抑制药物,C2,C2*不良反应,C2*不良反应#时间,C2*化疗药物,C2*放射治疗次数,C2*治疗性抗骨髓抑制药物,C2*肾小球滤过率,C2*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C2*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C2*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C2*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C2*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C2*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C2*血常规血生化_生化检查:肌酐(Crea)-静脉血,C2*血常规血生化_血常规:中性粒细胞计数-静脉血,C2*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C2*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C2*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C2*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C2*身体状况评价_PS评分,C2*预防性抗骨髓抑制药物,C3,C3*不良反应,C3*不良反应#时间,C3*化疗药物,C3*放射治疗次数,C3*治疗性抗骨髓抑制药物,C3*肾小球滤过率,C3*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C3*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C3*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C3*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C3*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C3*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C3*血常规血生化_生化检查:肌酐(Crea)-静脉血,C3*血常规血生化_血常规:中性粒细胞计数-静脉血,C3*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C3*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C3*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C3*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C3*身体状况评价_PS评分,C3*预防性抗骨髓抑制药物,C4,C4*不良反应,C4*不良反应#时间,C4*化疗药物,C4*放射治疗次数,C4*治疗性抗骨髓抑制药物,C4*肾小球滤过率,C4*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C4*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C4*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C4*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C4*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C4*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C4*血常规血生化_生化检查:肌酐(Crea)-静脉血,C4*血常规血生化_血常规:中性粒细胞计数-静脉血,C4*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C4*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C4*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C4*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C4*身体状况评价_PS评分,C4*预防性抗骨髓抑制药物,C5,C5*不良反应,C5*不良反应#时间,C5*化疗药物,C5*放射治疗次数,C5*治疗性抗骨髓抑制药物,C5*肾小球滤过率,C5*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C5*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C5*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C5*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C5*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C5*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C5*血常规血生化_生化检查:肌酐(Crea)-静脉血,C5*血常规血生化_血常规:中性粒细胞计数-静脉血,C5*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C5*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C5*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C5*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C5*身体状况评价_PS评分,C5*预防性抗骨髓抑制药物,C6,C6*不良反应,C6*不良反应#时间,C6*化疗药物,C6*放射治疗次数,C6*治疗性抗骨髓抑制药物,C6*肾小球滤过率,C6*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C6*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C6*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C6*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C6*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C6*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C6*血常规血生化_生化检查:肌酐(Crea)-静脉血,C6*血常规血生化_血常规:中性粒细胞计数-静脉血,C6*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C6*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C6*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C6*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C6*身体状况评价_PS评分,C6*预防性抗骨髓抑制药物,C7,C7*不良反应,C7*不良反应#时间,C7*化疗药物,C7*放射治疗次数,C7*治疗性抗骨髓抑制药物,C7*肾小球滤过率,C7*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C7*血常规血生化_生化检查:前白蛋白(PA)-静脉血,C7*血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血,C7*血常规血生化_生化检查:总胆红素(TBIL)-静脉血,C7*血常规血生化_生化检查:白蛋白(ALB)-静脉血,C7*血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血,C7*血常规血生化_生化检查:肌酐(Crea)-静脉血,C7*血常规血生化_血常规:中性粒细胞计数-静脉血,C7*血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血,C7*血常规血生化_血常规:白细胞计数(WBC#)-静脉血,C7*血常规血生化_血常规:血小板计数(PLT#)-静脉血,C7*血常规血生化_血常规:血红蛋白(Hb)-静脉血,C7*身体状况评价_PS评分,C7*预防性抗骨髓抑制药物,C8,C8*不良反应,C8*不良反应#时间,C8*化疗药物,C8*放射治疗次数,C8*治疗性抗骨髓抑制药物,C8*肾小球滤过率,C8*血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血,C8*

## keep variables of interest

In [11]:
dynamic_feature = [
    '肾小球滤过率',
    '血常规血生化_生化检查:天门冬氨酸氨基转移酶(AST)-静脉血',
    '血常规血生化_生化检查:总胆红素(TBIL)-静脉血',
    '血常规血生化_生化检查:前白蛋白(PA)-静脉血',
    '血常规血生化_生化检查:碱性磷酸酶(ALP)-静脉血',
    '血常规血生化_生化检查:白蛋白(ALB)-静脉血',
    '血常规血生化_生化检查:丙氨酸氨基转移酶(ALT)-静脉血',
    '血常规血生化_生化检查:肌酐(Crea)-静脉血',

    '血常规血生化_血常规:血小板计数(PLT#)-静脉血',
    '血常规血生化_血常规:血红蛋白(Hb)-静脉血',
    '血常规血生化_血常规:白细胞计数(WBC#)-静脉血',
    '血常规血生化_血常规:淋巴细胞计数(Lymph#)-静脉血',
    '血常规血生化_血常规:中性粒细胞计数-静脉血',
    
    '预防性抗骨髓抑制药物',

    '身体状况评价_PS评分'
]

In [12]:
state_feature_list = [
    '性别',
    '年龄',
    '人口学_首诊年龄(岁)',
    '体重',
    '身高',
    'ABS',
    'BMI',

    '治疗史_既往放疗',
    'C0*既往化疗',

    '合并慢性疾病_充血性心力衰竭',
    '合并慢性疾病_心绞痛',
    '合并慢性疾病_心肌梗死',
    '合并慢性疾病_慢阻肺',
    '合并慢性疾病_消化性溃疡',
    '合并慢性疾病_甲状腺疾病',
    '合并慢性疾病_糖尿病',
    '合并慢性疾病_自身免疫性疾病',
    '合并慢性疾病_获得性免疫缺陷综合征',

    '肿瘤状态_pM分期',
    '肿瘤状态_pN分期',
    '肿瘤状态_pT分期',

    'C0*预后影响指标'
]

In [13]:
cycle_feature_list = list()
for i in range(1,9):
    cycle = "C"+str(i)
    cycle_feature_list += [cycle+"*"+name for name in dynamic_feature+["interval"]]


In [14]:
ch_columns_list = ["patient_sn", "event", "首次不良反应#周期", "人口学_住院号", "年份", "cycle_num"] \
    + state_feature_list + cycle_feature_list
output_data = data[ch_columns_list]
# output_data
# output_data.to_csv("./output/datalong_024.csv", index = False, encoding = "utf_8_sig")

## rename variables

In [15]:
e_dynamic_feature = [
    'GFR',
    'AST',
    'TBIL',
    'PA',
    'ALP',
    'ALB',
    'ALT',
    'Crea',

    'PLT',
    'Hb',
    'WBC',
    'Lymph',
    'Neut',
    
    'prevention',

    'PS_score'
]

In [16]:
e_state_feature_list = [
    'gender',
    'age',
    'age_at_diagnosis',
    'weight',
    'height',
    'ABS',
    'BMI',

    'previous_radiation',
    'previous_chemotherapy',

    'congestive_heart_failure',
    'angina_pectoris',
    'myocardial_infarction',
    'copd',
    'peptic_ulcer',
    'thyroid_disease',
    'diabetes',
    'autoimmune_disease',
    'acquired_immune_deficiency_syndrome',

    'tumor_stage_pM',
    'tumor_stage_pN',
    'tumor_stage_pT',

    'PNI'
]

In [17]:
e_cycle_feature_list = list()
for i in range(1,9):
    cycle = "C"+str(i)
    e_cycle_feature_list += [cycle+"*"+name for name in e_dynamic_feature+["interval"]]

In [18]:
en_columns_list = ["patient_sn", "event", "event#cycle", "visit_sn", "visit_year", "cycle_num"] +\
    e_state_feature_list + e_cycle_feature_list
output_data.columns = en_columns_list
# output_data.to_csv("./output/datalong_en_024.csv", index = False)

In [19]:
ch2en_columns_name_dict = dict(zip(ch_columns_list, en_columns_list))
f_save = open('./output/ch2en_columns_name_dict.pkl', 'wb')
pickle.dump(ch2en_columns_name_dict, f_save)
f_save.close()

en2ch_columns_name_dict = dict(zip(en_columns_list, ch_columns_list))
f_save = open('./output/en2ch_columns_name_dict.pkl', 'wb')
pickle.dump(en2ch_columns_name_dict, f_save)
f_save.close()